**Notebook to convert eastings and northigns to Lat long and display them in a KML file that can be opened in Google Earth.**

what you will need to run this script:


*   the northings and eastings in order
*   the panel names in the corresponding order
*   The correct projection (in our case AZ Central State Plane meters
*   You will have to define the site name 'RCXXXX'
*   You will have to type in the year of the panel image
*   You will have to provide an output directory


In order to generate a rectification you need to develop a homography between the real-world (surveyed) GCP or panel coordinates and the UV (image) coordinates of those same GCPs or Panels from the image of the GCPs. In order to get the UV coordinates of the panels, the current workflow uses ENVI where you select the pixel which contains the GCPs to get the exact UV coordinates for the impts array. This aids in that by creating a KML file in google earth so you can see where the surveyed panels are on the bar, since the numbers on the panels are not visible in the image, it is also a good check to make sure you have your Eastings in northings in the correct order and that they line up with the real world location of the sandbar in question.

This script converts an array of map points np.array([Easting,Northing])
into a kml google earth file. The purpose of this is to visualize the 
surveyed panel locations.

In [39]:
# if not already installed, install simplekml which creates .kml files
#!pip install simplekml
# if not already installed, install pyproj which allows you to get the correct projection
#!pip install pyproj

from pyproj import CRS
import numpy as np
import simplekml
import os

In [40]:
'''This is an optional step if you are working on googleColab
if you are working on your local machine you can skip this but make sure to select an output directory
'''
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
site_name_string = input("Please type a string containing the site name RCXXXXXX:")

Please type a string containing the site name RCXXXXXX:RC0307Rf


In [42]:
year_string = input("Please type in the year the panels were placed: ")

Please type in the year the panels were placed: 2019


In [43]:
Output_dir = input("Please provide a pathway, where kml should be saved as a string with '\\\' or '/' as os.sep:")

Please provide a pathway, where kml should be saved as a string with '\' or '/' as os.sep:/content/drive/My Drive/Colab_Notebooks/Rectification/RC0307Rf


In [49]:
'''
** here is where you put an array of lists of the map-points np.array([Easting1, northing1],[Easting2, northing2],...)
** then you out a list of the panel names in the same order as strings ["p1","p2",...]
** you can also change the size of the buffer here, by default its set at 30 meters
 the chosen buffer value is subtracted from the minNorthing and the minEasting, and added to the maxNorthing and maxEasting
 the buffer gives you a MaxNE and a MinNE which will be the size of your rectified image after rectification
'''
#####################################
site = site_name_string
year = int(year_string)
extent_buffer_in_meters = 30



mappts = np.array([
[219542.644,611803.324],
[219542.917,611790.645],
[219540.663,611778.078],
[219522.509,611775.796],
[219520.759,611764.155],
[219515.123,611754.640],
[219529.343,611747.805],
[219515.438,611733.189],
[219541.495,611757.829],
[219542.715,611735.480],
[219549.039,611767.098],
[219571.880,611771.835]
])

extent_buffer_in_meters = 30



panels = [
'p13',
'p30',
'p23',
'p16',
'p17',
'p21',
'p24',
'p18',
'p19',
'p14',
'p15',
'p20',
]

# create the point names by concatenating the panel name to the site name and year
point_names = []
for point in panels:
    name = str(point + '_'+ year_string + "_" + site )
    point_names.append(name)
print(point_names)
##################################

['p13_2019_RC0307Rf', 'p30_2019_RC0307Rf', 'p23_2019_RC0307Rf', 'p16_2019_RC0307Rf', 'p17_2019_RC0307Rf', 'p21_2019_RC0307Rf', 'p24_2019_RC0307Rf', 'p18_2019_RC0307Rf', 'p19_2019_RC0307Rf', 'p14_2019_RC0307Rf', 'p15_2019_RC0307Rf', 'p20_2019_RC0307Rf']


In [50]:
# espg:26949 is Arizona State Plane Central Meters
cs2cs_args = CRS.from_epsg(26949)
# cs2cs_args = CRS(init="epsg:26949")
# #print("using projection {cs3cs_args}")
#trans =  pyproj.Proj(init=cs2cs_args)
trans = pyproj.Proj(cs2cs_args)

In [51]:
# create the NEmax and NEmin
E_max = np.max(mappts[:,0]) + extent_buffer_in_meters
E_min = np.min(mappts[:,0]) - extent_buffer_in_meters
N_max = np.max(mappts[:,1]) + extent_buffer_in_meters
N_min = np.min(mappts[:,1]) - extent_buffer_in_meters


print(E_max,N_max)
print(E_min,N_min)

ENminlon, ENminLat = trans(E_min,N_min,inverse = True)
ENmaxlon, ENmaxlat = trans(E_max,N_max,inverse = True)
ENmax_coords = (ENmaxlon, ENmaxlat)
ENmin_coords = (ENminlon, ENminLat)

219601.88 611833.324
219485.123 611703.189


In [52]:
# convert points to Lat, long
coords = []
for point in mappts:
    lon, lat = trans(point[0],point[1],inverse = True)
    coords.append((lon,lat))
print(coords)    

[(-111.84763569412608, 36.51633009244613), (-111.84763274750169, 36.51621582179101), (-111.84765801455917, 36.51610257685405), (-111.84786072659567, 36.51608212752322), (-111.84788035861826, 36.51597722482215), (-111.84794336165332, 36.51589150759663), (-111.84778464691585, 36.51582981590134), (-111.84794001556021, 36.51569817942503), (-111.84764888713146, 36.5159200783115), (-111.8476354444614, 36.515718651102844), (-111.84756458246197, 36.51600356597832), (-111.84730951936649, 36.51604610996363)]


In [53]:
# add points to a kml file
kml = simplekml.Kml()
for i in range(len(point_names)):
    kml.newpoint(name = point_names[i], coords=[(coords[i])] )

NE_max_name = str(site + '_NE_max')
NE_min_name = str(site + '_NE_min')
kml.newpoint(name = NE_max_name, coords =[(ENmax_coords)] )
kml.newpoint(name = NE_min_name, coords =[(ENmin_coords)] )

kml_name = str('panels' + year_string +"_" +site +'.kml')
# export the kml file 
kml.save(Output_dir+ os.sep + kml_name)
print(f'kml for site {site} created, filename is {kml_name}')
print(f'NE max point = {ENmax_coords}, NE min point = {ENmin_coords}')

kml for site RC0307Rf created, filename is panels2019_RC0307Rf.kml
NE max point = (-111.8469740653941, 36.51660008144793), NE min point = (-111.84827872605499, 36.515428000327645)
